In [1]:
import os
os.makedirs('../data',exist_ok=True)
import torchvision
mnist_train = torchvision.datasets.MNIST(root = "../data",train = True,download=True)

c:\Bib\envs\azml_training\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np
from tqdm import tqdm

NMAX = 1000 # We'll keep 1000 images per classes
NClasses = len(mnist_train.classes)

indices = np.arange(len(mnist_train))
np.random.shuffle(indices)
selection = {clsName:[] for clsName in mnist_train.classes}
for i in tqdm(indices):
    img_pil,clsID = mnist_train[i]
    clsName = mnist_train.classes[clsID]
    if len(selection[clsName])<NMAX:
        # We convert in RGB (optionnal)
        selection[clsName].append(img_pil.convert('RGB')) 
    if sum([len(indices) for indices in selection.values()])==NMAX*NClasses:
        print('Selection Completed !')
        break

 19%|█▊        | 11191/60000 [00:00<00:03, 12817.96it/s]

Selection Completed !


In [3]:
import shutil
import os
from PIL import Image as PIL_Image

# We'll save images in a directory with class as name.
destination_root = "./data/MNIST_FLAT/"
shutil.rmtree(destination_root,ignore_errors=True)
os.makedirs(destination_root,exist_ok=True)
for clsName in selection:
    dest_dir = os.path.join(destination_root,clsName)
    os.makedirs(dest_dir,exist_ok=True)
    images = selection[clsName]
    for i,img in tqdm(enumerate(images),total=len(images)):
        im_name = f"{clsName}_{i}.jpg"
        im_path = os.path.join(dest_dir,im_name)
        img:PIL_Image
        img.save(im_path)

 60%|█████▉    | 595/1000 [00:00<00:00, 596.78it/s]

In [5]:
destination_root = "./data/MNIST_FLAT/"

from azure.storage.blob import ContainerClient,BlobServiceClient
connection_string = "<CONNECTION STRING HERE>"
container_name = "mnist-data"
container_client = ContainerClient.from_connection_string(connection_string,container_name)

assert container_client.exists()

for clsName in selection:
    im_dir = os.path.join(destination_root,clsName)
    tbar = tqdm(os.listdir(im_dir))
    for imgName in tbar:
        imgRelativePath = os.path.join(clsName,imgName)
        imgPath = os.path.join(destination_root,imgRelativePath)
        with open(imgPath,'rb') as img_data:
            container_client.upload_blob(
                name=imgRelativePath,
                data=img_data,
                overwrite=True)
        tbar.set_postfix_str(f'Image {imgRelativePath}')

100%|██████████| 1000/1000 [00:42<00:00, 23.71it/s, Image 9 - nine\9 - nine_999.jpg]
